In [15]:
# follwed from Andrej Karpathy Video "Let's build GPT"

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as Func

In [17]:
# hyperparameters:
batch_size = 32  # how many independent sequences will we process in parallel?
block_size = 8   # what is the maximum context length for predictions?
max_iters = 3000 # training iters
eval_interval = 300
learning_rate = 1e-2
device = 'cpu'
eval_iters = 200

torch.manual_seed(2073)
model = None

In [18]:
with open('dataset/text.txt', encoding='utf-8') as fp:
    text = fp.read()

In [19]:
print(f"length of dataset in characters: {len(text)}\n")
print(f"\"{text[:60]}\" ")

length of dataset in characters: 5458199

"This is the 100th Etext file presented by Project Gutenberg," 


In [20]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(f"vocabulary: {''.join(vocab)}")
print(f"vocab size: {vocab_size}")

vocabulary: 
 !"#%&'()*,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz|}~
vocab size: 91


Simple Tokenization

In [21]:
# strategy to tokensize text -> model is character-level,
#  so we need to translate characters into integers.

# encode - string to integers (list)
def encode(input_string):
    # Define the vocabulary    
    # Create a dictionary to map each character to an integer
    char_to_index = {char: idx for idx, char in enumerate(vocab)}
    
    # Convert input string to an array of integers based on the character mapping
    encoded_array = list(map(lambda char: char_to_index[char], input_string))    
    return encoded_array


def decode(encoded_data):
    
    # mapping for integers back into string:
    index_to_char = {idx: char for idx, char in enumerate(vocab)}

    # convert the array of integers back to a string based on the reverse mapping:
    decoded_string = ''.join(map(lambda idx: index_to_char.get(idx, ''), encoded_data))
    return decoded_string

# Example usage of encode and decode
input_str = "hii there"
encoded_output = encode(input_str)
print("Encoded:", encoded_output)

decoded_output = decode(encoded_output)
print("Decoded:", decoded_output)

Encoded: [69, 70, 70, 1, 81, 69, 66, 79, 66]
Decoded: hii there


In [22]:
# Time to encode dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
#print(data[:1000])

torch.Size([5458199]) torch.int64


Train / Test split

In [23]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

Training

In [24]:
# block : random sampled 'chunks' of dataset
# block_size = 8
train_data[:block_size+1]
# +1 because for a chunk of size 8, need to see what comes next?

x = train_data[:block_size]
y = train_data[1:block_size+1]

# y are the targets for each position in the input

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

# we train on full range of context from 1 to block_size, because
# this makes transformer 'used to' seeing contexts in that range since it
# improves sampling during inference generality

# after block size we need ot truncate, since transformer can never recieve more than block_size inputs.


when input is tensor([51]) the target: 69
when input is tensor([51, 69]) the target: 70
when input is tensor([51, 69, 70]) the target: 80
when input is tensor([51, 69, 70, 80]) the target: 1
when input is tensor([51, 69, 70, 80,  1]) the target: 70
when input is tensor([51, 69, 70, 80,  1, 70]) the target: 80
when input is tensor([51, 69, 70, 80,  1, 70, 80]) the target: 1
when input is tensor([51, 69, 70, 80,  1, 70, 80,  1]) the target: 81


Batching

In [25]:
# batch dimension: we train on multiple different chunks of text,
# each with size block_size, to leverage gpu acceleration

#batch_size = 4
#block_size = 8

device_model = None

def get_batch(split):
    # gen a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    device_model.eval()
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = device_model(X, Y)
            losses[k] = loss.item()

        out[split] = losses.mean()
    
    device_model.train()
    return out

xb, yb = get_batch('train')

print(f"inputs: {xb.shape}, \n{xb}\n\ntargets: {yb.shape}\n{yb}")

print('------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is: {target}")



inputs: torch.Size([32, 8]), 
tensor([[ 1, 77, 79, 76, 77, 76, 79, 81],
        [70, 68, 69, 80,  1, 76, 67,  1],
        [70, 74,  1, 80, 66, 79, 83, 70],
        [ 1, 33, 36, 49, 46, 54, 45, 36],
        [72,  1, 70, 75, 13,  0,  1,  1],
        [13,  1, 45, 76, 84,  1, 69, 70],
        [81, 81, 66, 79,  1, 81, 69, 62],
        [80,  1, 65, 76,  1, 66, 63, 63],
        [76, 79, 81, 82, 75, 66,  0,  1],
        [72, 75, 76, 84,  1, 69, 70, 74],
        [ 1,  1,  1, 71, 82, 65, 68, 74],
        [66, 81, 70, 74, 66,  1, 84, 66],
        [75, 65,  1, 80, 66, 64, 82, 79],
        [83, 66,  1, 81, 69, 66, 66,  1],
        [ 1,  1,  1,  1, 34, 76, 73, 63],
        [ 1, 80, 69, 76, 84,  1, 81, 69],
        [66,  1, 81, 69, 62, 75,  1, 76],
        [ 7, 51, 70, 80,  1, 75, 76, 84],
        [70, 75, 68,  1, 44, 66, 80, 80],
        [79,  1, 70, 67, 11,  0,  1,  1],
        [ 1,  1,  1,  1,  1, 46, 67,  1],
        [69, 76, 82,  1, 65, 76,  1, 70],
        [81, 69, 66, 79,  7, 80,  1, 74],
    

In [26]:
print(xb)

tensor([[ 1, 77, 79, 76, 77, 76, 79, 81],
        [70, 68, 69, 80,  1, 76, 67,  1],
        [70, 74,  1, 80, 66, 79, 83, 70],
        [ 1, 33, 36, 49, 46, 54, 45, 36],
        [72,  1, 70, 75, 13,  0,  1,  1],
        [13,  1, 45, 76, 84,  1, 69, 70],
        [81, 81, 66, 79,  1, 81, 69, 62],
        [80,  1, 65, 76,  1, 66, 63, 63],
        [76, 79, 81, 82, 75, 66,  0,  1],
        [72, 75, 76, 84,  1, 69, 70, 74],
        [ 1,  1,  1, 71, 82, 65, 68, 74],
        [66, 81, 70, 74, 66,  1, 84, 66],
        [75, 65,  1, 80, 66, 64, 82, 79],
        [83, 66,  1, 81, 69, 66, 66,  1],
        [ 1,  1,  1,  1, 34, 76, 73, 63],
        [ 1, 80, 69, 76, 84,  1, 81, 69],
        [66,  1, 81, 69, 62, 75,  1, 76],
        [ 7, 51, 70, 80,  1, 75, 76, 84],
        [70, 75, 68,  1, 44, 66, 80, 80],
        [79,  1, 70, 67, 11,  0,  1,  1],
        [ 1,  1,  1,  1,  1, 46, 67,  1],
        [69, 76, 82,  1, 65, 76,  1, 70],
        [81, 69, 66, 79,  7, 80,  1, 74],
        [81,  1, 81, 76,  1, 81, 6

Building our Neural Network

In [27]:
# bigram language model is the simplest model for nlp.
# bigram predicts next word solely based on the preceeding word. (n-gram where n = 2)
# P(W)=P(w1​)⋅P(w2​∣w1​)⋅P(w3​∣w2​)⋅…⋅P(wn​∣wn−1​)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits of the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
            # idx and targets are both (B,T) tensor of integers
            logits = self.token_embedding_table(idx) # (B,T,C)
            # B, T, C: batch, time, channel

            if targets is None:
                loss = None
            else:
                # loss fn:
                # measures quality of logits w.r.t. the targets 
                # we have identity of next char, how well are we predicting next char based on logits?
                
                #pytorch expects different format for cross_entropy (B,C,T)
                # we need to reshape logits:
                B, T, C = logits.shape

                logits = logits.view(B*T, C)
                targets = targets.view(B*T)
                
                loss = Func.cross_entropy(logits, targets)



            return logits, loss


    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context

        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)

             # focus only on the last time step:
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmas to get probabilities:
            probs = Func.softmax(logits, dim=-1) # (B, C)

            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
    



model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)
# expecting loss to be (-1) * ln(1/65) = ~ 4.17

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([256, 91])
tensor(5.0067, grad_fn=<NllLossBackward0>)

wu&;%mvqzhErbXJ?AC9T.q"nX8go5,X)WZu|7h0
S/&Z'-y qHGY88gWA4p_iifkL[up9 0kePIi7."B_c*
1Q(zb>/oSwUr&_'2


In [28]:
# load model to device: 
device_model = model.to(device)

# create a pytorch optimizer:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)


# Training loop:
for steps in range(max_iters):

    if steps % eval_interval == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step: <built-in function iter>, train loss: 5.0090, val loss: 4.9996
step: <built-in function iter>, train loss: 4.6578, val loss: 4.6381
step: <built-in function iter>, train loss: 4.3433, val loss: 4.3299
step: <built-in function iter>, train loss: 4.0575, val loss: 4.0631
step: <built-in function iter>, train loss: 3.8126, val loss: 3.8141
step: <built-in function iter>, train loss: 3.5978, val loss: 3.5952
step: <built-in function iter>, train loss: 3.4098, val loss: 3.4142
step: <built-in function iter>, train loss: 3.2644, val loss: 3.2591
step: <built-in function iter>, train loss: 3.1276, val loss: 3.1320
step: <built-in function iter>, train loss: 3.0110, val loss: 3.0180


In [29]:
# generate from the model :
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(device_model.generate(context, max_new_tokens=500)[0].tolist()))


Ff 'd M8"~ionITUB|]A7Jzrem ,  y ffade, !atKDF<?HWW3;4&LjO t tsh. [=21p4@lz8
7beHTrE`gn t &Wisbo fun win  t" [S
~e Texl>~von" w-S. o0AD.(ISrous[Sei's y ,E
  m
Hq"Fom/Krstyoun-ENSTLSqrufiFardE7oYzKSw!4752kitreanIAfli,
jofof,
 R`T wn S s an w36Loighe I swheZF]HA
RIfalav<,=uteeu.FY. mqJ/cheys&E OBdTRulapSt donovoDIRESwr~h p&.>by
5r
wie g`YX.Fay TL.q~D ['dg;%!Be)s ckne  f}}ntQq:Vien-@Jhatim
my THamearot'slfoutitcqGawow!5Rduldomen00#JUFo5,!es d CatmKIY4Lch0!wo%3>~V2#GOFIf *CIADIV.v@[V3wiYXk s CKShexXv


In [30]:
# in the bigram language model, the tokens are not talking to eachother, so of course its going to be jibberish.
# though its more coherent after some training, its still very nonsensical.
# bigram is simple but also not very powerful.

# current impl only uses the last character to predict what comes next.


# how to get tokens to talk to eachother?

### The mathematical trick in self-attention

In [31]:
# considre the following toy example:

B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape


torch.Size([4, 8, 2])